<a href="https://colab.research.google.com/github/kisharnath/Tools-in-DataScience/blob/main/Copy_of_TDS_Kumu_GA_6a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DO NOT MODIFY THIS FILE DIRECTLY.
## First save a copy of this file to your Google Drive and work on the copy instead.

In [1]:
!nvidia-smi

Sun Mar 17 15:28:47 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix

# Load IMDb datasets
!curl --silent -C - -o name.basics.tsv.gz https://datasets.imdbws.com/name.basics.tsv.gz
!curl --silent -C - -o title.akas.tsv.gz https://datasets.imdbws.com/title.akas.tsv.gz
!curl --silent -C - -o title.basics.tsv.gz https://datasets.imdbws.com/title.basics.tsv.gz
!curl --silent -C - -o title.crew.tsv.gz https://datasets.imdbws.com/title.crew.tsv.gz
!curl --silent -C - -o title.episode.tsv.gz https://datasets.imdbws.com/title.episode.tsv.gz
!curl --silent -C - -o title.principals.tsv.gz https://datasets.imdbws.com/title.principals.tsv.gz
!curl --silent -C - -o title.ratings.tsv.gz https://datasets.imdbws.com/title.ratings.tsv.gz

In [2]:
title = pd.read_csv('title.basics.tsv.gz', sep='\t', low_memory=False).set_index('tconst')[['titleType', 'primaryTitle', 'startYear', 'genres']]
title

,titleType,primaryTitle,startYear,genres
tconst,,,,
tt0000001,short,Carmencita,1894,"Documentary,Short"
tt0000002,short,Le clown et ses chiens,1892,"Animation,Short"
tt0000003,short,Pauvre Pierrot,1892,"Animation,Comedy,Romance"
tt0000004,short,Un bon bock,1892,"Animation,Short"
tt0000005,short,Blacksmith Scene,1893,"Comedy,Short"
...,...,...,...,...
tt9916848,tvEpisode,Episode #3.17,2009,"Action,Drama,Family"
tt9916850,tvEpisode,Episode #3.19,2010,"Action,Drama,Family"
tt9916852,tvEpisode,Episode #3.20,2010,"Action,Drama,Family"


In [3]:
!rm title.principals.tsv.gz  # Remove the corrupted file
!wget -q https://datasets.imdbws.com/title.principals.tsv.gz -O /content/title.principals.tsv.gz


In [4]:
!gzip -dk title.principals.tsv.gz

gzip: title.principals.tsv already exists; do you wish to overwrite (y or n)? ^C


In [5]:
title = title.sort_values(by='startYear')

In [6]:
title.head()

,titleType,primaryTitle,startYear,genres
tconst,,,,
tt3155794,short,Passage de Venus,1874,"Documentary,Short"
tt16763774,short,Dzing. Boom. Boom!,1877,"Animation,Short"
tt16763740,short,Skipping Rope,1877,"Animation,Short"
tt14495706,short,La Rosace Magique,1877,"Animation,Short"
tt16763674,short,The Tight-rope Dance,1877,"Animation,Short"


In [7]:
title=title[(title["startYear"] >= str(2005)) & (title['titleType']=='movie')]

In [8]:
title.shape

(390614, 4)

In [9]:
title.dtypes

titleType       object
primaryTitle    object
startYear       object
genres          object
dtype: object

In [10]:
title["startYear"].replace('\\N', 0000, inplace=True)

In [11]:
title = title[title["startYear"] != '\\N']

In [12]:
title.info()

<class 'pandas.core.frame.DataFrame'>
Index: 390614 entries, tt0441902 to tt15440956
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   titleType     390614 non-null  object
 1   primaryTitle  390614 non-null  object
 2   startYear     390614 non-null  object
 3   genres        390614 non-null  object
dtypes: object(4)
memory usage: 14.9+ MB


In [13]:
title[title.genres.values == 'Bollywood']


,titleType,primaryTitle,startYear,genres
tconst,,,,


In [14]:
# Define chunk size
chunk_size = 5000

# Initialize an empty list to store chunks
chunks = []

# Iterate over the dataset in chunks
for chunk in pd.read_csv('title.principals.tsv.gz', sep='\t', chunksize=chunk_size, low_memory=False):
    # Process each chunk as needed
    processed_chunk = chunk.set_index('tconst')[['nconst', 'category']]
    chunks.append(processed_chunk)

# Concatenate the processed chunks into a single DataFrame
cast = pd.concat(chunks)

In [15]:
! wget -q https://datasets.imdbws.com/title.principals.tsv.gz

In [ ]:
import pandas as pd

cast = pd.read_csv('title.principals.tsv.gz', sep='\t', low_memory=False).set_index('tconst')[['nconst', 'category']]

In [ ]:
# Restrict data to just a single region (e.g. IN, US, etc)
# This loads the region for each title

region = pd.read_csv('title.akas.tsv.gz', sep='\t', low_memory=False,nrows=10000).set_index('titleId')['region']
region.value_counts().head (20)

In [ ]:
merged_data = pd.merge(title, region, left_on='tconst', right_on='titleId', how='inner')

In [ ]:
pd.merge

In [ ]:
merged_data

In [ ]:
title[
        (title['titleType'] == 'movie') &
        (title['startYear'].astype(str).str.isnumeric()) &
        (title['startYear'].astype(int) >= 2005) &
        (title['genres'].str.contains('Bollywood', na=False))
    ]

In [ ]:
# Load the cast of each film
import pandas as pd
cast = pd.read_csv('title.principals.tsv.gz',nrows=10000, sep='\t')[['tconst', 'nconst', 'category']]
# Only consider actors, not directors, composers, etc. Shrinks data to about 40%

cast = cast [cast.category.isin({'actor', 'actress'})]
cast.head()

In [ ]:
# Function to get actor pairs for Bollywood movies from 1995 onwards
def get_pairs_bollywood(lang=None, min_acted=0, min_pairings=0):
    # Filter Bollywood movies released from 2005 onwards with 'Bollywood' genre
    bollywood_movies = title[
        (title['titleType'] == 'movie') &
        (title['startYear'].astype(str).str.isnumeric()) &
        (title['startYear'].astype(int) >= 2005) &
        (title['genres'].str.contains('Bollywood', na=False))
    ]

    print("The size is ", len(bollywood_movies))

    # Extract actor and movie IDs for Bollywood movies
    bollywood_movie_ids = bollywood_movies.index
    bollywood_actors = cast[
        (cast['tconst'].isin(bollywood_movie_ids)) &
        (cast['category'].isin(['actor', 'actress']))
    ]

    graph = bollywood_actors

    name_freq = graph['nconst'].value_counts()

    top_names = name_freq[name_freq >= min_acted]
    top_actors = graph[graph['nconst'].isin(top_names.index)]

    p = top_actors.copy()
    p['title'] = p['tconst'].astype('category')
    p['name'] = p['nconst'].astype('category')

    row = p['title'].cat.codes.values
    col = p['name'].cat.codes.values
    data = np.ones(len(p), dtype='int')

    matrix = csr_matrix((data, (row, col)))
    square = matrix.T * matrix
    square.setdiag(0)
    square = square.tocoo()

    pairs = pd.DataFrame({
        'row': square.row,
        'col': square.col,
        'n': square.data
    })
    pairs = pairs[pairs.n >= min_pairings].reset_index(drop=True)
    return pairs, top_names.reindex(p['name'].cat.categories)

def lookup_bollywood(pairs, cat):
    pairs = pd.concat([
        pairs,
        cat.iloc[pairs.row].reset_index(drop=True),
        cat.iloc[pairs.col].reset_index(drop=True),
    ], axis=1)
    pairs = pairs.drop(columns=['row', 'col'])
    pairs.columns = ['count', 'name1', 'year1', 'name2', 'year2']
    return pairs.sort_values('count', ascending=False)



In [ ]:
import numpy as np
get_pairs_bollywood(lang='IN', min_acted=0, min_pairings=0)

In [ ]:
title = title[title['startYear'].str.find("\\N")== -1]

In [ ]:
title["startYear"] = title.startYear.values.astype('int')

In [ ]:
pairs, cat = get_pairs_bollywood(min_acted=10, min_pairings=3)

In [ ]:
Forkumu = lookup_hollywood(pairs)

In [ ]:
from google.colab import files
files.download("data.csv")

In [ ]:
import wikipedia
a=wk.page("Business Intelligence")
a

In [ ]:
import numpy as np


In [ ]:
j=1000
i=6

In [ ]:
x= np.random.randn(j)
print(x)
print(3*x)
print(6+3*x)

In [ ]:
data=np.array([5,7,2,1])

In [ ]:
data

In [ ]:
data.reshape(2,2)

In [ ]:
p = int(input())
r = int(input())
n = int(input())
a = p*(1 + r/100)**n

print(a-p)

In [ ]:
# Import necessary modules
from qgis.core import QgsVectorLayer, QgsProject

# Path to the shapefile for Administrative Areas of the United States
shapefile_path = '/path/to/your/shapefile.shp'

# Load the shapefile as a vector layer
layer = QgsVectorLayer(shapefile_path, 'Administrative Areas', 'ogr')

# Check if the layer is valid
if not layer.isValid():
    print("Layer failed to load!")

# Get the field index for the county name field
county_field_index = layer.fields().indexFromName('COUNTY_NAME')

# Define the county name you want to find the area for
target_county = 'Yellowstone'

# Filter features based on county name
expression = QgsExpression(f'"COUNTY_NAME" = \'{target_county}\'')
it = layer.getFeatures(QgsFeatureRequest(expression))

# Initialize total area
total_area = 0

# Iterate over filtered features and calculate total area
for feature in it:
    total_area += feature.geometry().area()

# Convert area to square kilometers (assuming the CRS of the shapefile is in meters)
total_area_sq_km = total_area / 1000000

# Print the result
print(f"The area of {target_county} County in Montana is approximately {total_area_sq_km:.2f} sq.km")

# Don't forget to remove the layer from the project if it's no longer needed
QgsProject.instance().removeMapLayer(layer)
